In [1]:
import torch
from torch import nn,optim
from torch.nn import functional as F
import pandas as pd
import numpy as np
import random
import pickle
import logging
from torch.autograd import Variable
USE_CUDA = torch.cuda.is_available()

random.seed(2020)
np.random.seed(2020)
torch.manual_seed(2020)
if USE_CUDA:
    torch.cuda.manual_seed(2020)
# set cuda
gpu = 0
use_cuda = gpu >= 0 and torch.cuda.is_available()
if use_cuda:
    torch.cuda.set_device(gpu)
    device = torch.device("cuda", gpu)
else:
    device = torch.device("cpu")
logging.info("Use cuda: %s, gpu id: %d.", use_cuda, gpu)

In [2]:
# file_path = 'data_example/parameters.pkl'
# with open(file_path, 'rb') as f:  
#     parameters = pickle.loads(f.read())

In [3]:
# parameters.keys()

In [4]:
# 将train_data和test_data读取存入list中
def ReadTxtName(rootdir):
    lines = []
    with open(rootdir, 'r') as file_to_read:
        while True:
            line = file_to_read.readline()
            if not line:
                break
            line = line.strip('\n')
            line = list(eval(line))
            lines.append(line)
    return lines

In [5]:
train_data_path = 'data_example/train_data.txt'
test_data_path = 'data_example/test_data.txt'

In [6]:
train_set = ReadTxtName(train_data_path)
test_set = ReadTxtName(test_data_path)

In [7]:
train_data = pd.DataFrame(columns=['user_id', 'item_seq_temp','cat_list','time_list','time_last_list','time_now_list','position_list','target','item_seq_len'],data = train_set)
test_data = pd.DataFrame(columns=['user_id', 'item_seq_temp','cat_list','time_list','time_last_list','time_now_list','position_list','target','item_seq_len'],data = test_set)

In [8]:
train_data.head()

,user_id,item_seq_temp,cat_list,time_list,time_last_list,time_now_list,position_list,target,item_seq_len
0,3522,"[118873, 190989, 73693, 354311, 73693, 354311,...","[571, 894, 482, 482, 482, 482, 482, 202, 330, ...","[398008, 398008, 398104, 398104, 398104, 39810...","[0, 0, 96, 0, 0, 0, 0, 864, 48, 0, 0, 0, 0, 0,...","[1032, 1032, 936, 936, 936, 936, 936, 72, 24, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[125165, 482, 399040]",26
1,3839,"[298668, 297576, 305398, 107015, 32892, 76809,...","[636, 506, 339, 339, 636, 25, 339, 339, 25, 94...","[401704, 401704, 401704, 401704, 401704, 40170...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 4...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[168523, 943, 401752]",23
2,3719,"[211347, 153658, 7223, 81509, 19699, 81509, 10...","[719, 719, 613, 352, 1333, 352, 352, 1333, 120...","[400216, 400216, 400480, 400552, 400552, 40057...","[0, 0, 264, 72, 0, 24, 0, 0, 0, 24, 24, 288, 0...","[1008, 1008, 744, 672, 672, 648, 648, 648, 648...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[329307, 344, 401224]",17
3,4362,"[95057, 68099, 161481, 366635, 150257, 303230,...","[1203, 704, 704, 704, 704, 704, 704, 704, 704,...","[399184, 400192, 400192, 400240, 400240, 40024...","[0, 1008, 0, 48, 0, 0, 0, 0, 0, 384, 0, 0, 72,...","[2520, 1512, 1512, 1464, 1464, 1464, 1464, 146...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[146956, 232, 401704]",19
4,1104,"[228127, 202790, 37771, 210693, 316530, 186423...","[537, 381, 194, 962, 194, 301, 194, 194, 1098,...","[397672, 397984, 398008, 398536, 398656, 39865...","[0, 312, 24, 528, 120, 0, 0, 0, 24, 312, 0, 0,...","[3864, 3552, 3528, 3000, 2880, 2880, 2880, 288...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[111155, 67, 401536]",18


In [9]:
# 对train_data和test_data进行padding,即item_count+1作为padding_idx
# item_id,user_id,cat_list_id进行了重新编码
train_data['target_item'] = train_data['target'].apply(lambda x:x[0]) 
test_data['target_item'] = test_data['target'].apply(lambda x:x[0]) 

In [10]:
def pad(data):
    for i in data.columns:
        if i == 'item_seq_temp' or i =='cat_list' or i=='position_list':
            data[i] = data[i].apply(lambda x:x+[x[-1]]*(input_length-len(x))if len(x)<input_length else x[:input_length])

In [11]:
input_length = 50
pad(train_data)

In [12]:
pad(test_data)

In [13]:
train_data.head()

,user_id,item_seq_temp,cat_list,time_list,time_last_list,time_now_list,position_list,target,item_seq_len,target_item
0,3522,"[118873, 190989, 73693, 354311, 73693, 354311,...","[571, 894, 482, 482, 482, 482, 482, 202, 330, ...","[398008, 398008, 398104, 398104, 398104, 39810...","[0, 0, 96, 0, 0, 0, 0, 864, 48, 0, 0, 0, 0, 0,...","[1032, 1032, 936, 936, 936, 936, 936, 72, 24, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[125165, 482, 399040]",26,125165
1,3839,"[298668, 297576, 305398, 107015, 32892, 76809,...","[636, 506, 339, 339, 636, 25, 339, 339, 25, 94...","[401704, 401704, 401704, 401704, 401704, 40170...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 4...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[168523, 943, 401752]",23,168523
2,3719,"[211347, 153658, 7223, 81509, 19699, 81509, 10...","[719, 719, 613, 352, 1333, 352, 352, 1333, 120...","[400216, 400216, 400480, 400552, 400552, 40057...","[0, 0, 264, 72, 0, 24, 0, 0, 0, 24, 24, 288, 0...","[1008, 1008, 744, 672, 672, 648, 648, 648, 648...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[329307, 344, 401224]",17,329307
3,4362,"[95057, 68099, 161481, 366635, 150257, 303230,...","[1203, 704, 704, 704, 704, 704, 704, 704, 704,...","[399184, 400192, 400192, 400240, 400240, 40024...","[0, 1008, 0, 48, 0, 0, 0, 0, 0, 384, 0, 0, 72,...","[2520, 1512, 1512, 1464, 1464, 1464, 1464, 146...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[146956, 232, 401704]",19,146956
4,1104,"[228127, 202790, 37771, 210693, 316530, 186423...","[537, 381, 194, 962, 194, 301, 194, 194, 1098,...","[397672, 397984, 398008, 398536, 398656, 39865...","[0, 312, 24, 528, 120, 0, 0, 0, 24, 312, 0, 0,...","[3864, 3552, 3528, 3000, 2880, 2880, 2880, 288...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[111155, 67, 401536]",18,111155


In [14]:
# 获得train_data和test_data
columns = ['user_id','item_seq_temp','cat_list','position_list','item_seq_len','target_item']
train_data= train_data[columns]
test_data = test_data[columns]

In [15]:
# 方便处理数据，转换为numpy格式
user_id = np.array(train_data['user_id'].tolist(),dtype = np.int32)
item_seq_temp = np.array(train_data['item_seq_temp'].tolist(),dtype = np.int32)
cat_list = np.array(train_data['cat_list'].tolist(),dtype = np.int32)
position_list = np.array(train_data['position_list'].tolist(),dtype = np.int32)
item_seq_len = np.array(train_data['item_seq_len'].tolist(),dtype = np.int32)
target_item = np.array(train_data['target_item'].tolist(),dtype = np.int32)

# 生成batch

In [16]:
def get_batch(x_user_id,x_item_seq_temp,x_cat_list,x_position_list,x_item_seq_len,y,batch_size,shuffle = True):
    assert x_user_id.shape[0] == y.shape[0]
    if shuffle:
        shuffled_index = np.random.permutation(y.shape[0])
        x_user_id,x_item_seq_len,x_cat_list,x_position_list,x_item_seq_len = x_user_id[shuffled_index],x_item_seq_len[shuffled_index],x_cat_list[shuffled_index],x_position_list[shuffled_index],x_item_seq_len[shuffled_index]
        y = y[shuffled_index]
    
    n_batches = int(x_user_id.shape[0]/batch_size)
    for i in range(n_batches-1):
        x_user_id_batch = x_user_id[i*batch_size:(i+1)*batch_size]
        x_item_seq_temp_batch = x_item_seq_temp[i*batch_size:(i+1)*batch_size]
        x_cat_list_batch = x_cat_list[i*batch_size:(i+1)*batch_size]
        x_position_list_batch = x_position_list[i*batch_size:(i+1)*batch_size]
        x_item_seq_len_batch = x_item_seq_len[i*batch_size:(i+1)*batch_size]
        y_batch = y[i*batch_size:(i+1)*batch_size]
        yield x_user_id_batch,x_item_seq_temp_batch,x_cat_list_batch,x_position_list_batch,x_item_seq_len_batch,y_batch

# 构造模型

In [33]:
class GetEmbedding(nn.Module):
    def __init__(self,parameter_path,input_length,embedding_dim):
        super(GetEmbedding,self).__init__()
        self.parameter_path = parameter_path
        self.parameters = self.get_parameter(self.parameter_path)
        self.user_count = self.parameters['user_count']
        self.item_count = self.parameters['item_count']
        self.category_count = self.parameters['category_count']
        self.input_length = input_length
        self.embedding_dim = embedding_dim
        self.user_id_embedding = nn.Embedding(self.user_count+3,self.embedding_dim)
        self.item_list_embedding = nn.Embedding(self.item_count+3,self.embedding_dim)
        self.item_list_embedding_weight = self.item_list_embedding.weight
        self.category_list_embedding = nn.Embedding(self.category_count+3,self.embedding_dim)
        self.position_list_embeddig = nn.Embedding(self.input_length,self.embedding_dim)
        self.apply(self.init_weights)
    def init_weights(self, module):
        if isinstance(module, nn.Embedding):
            nn.init.xavier_uniform_(module.weight)
        
    def get_parameter(self,file_path):
        with open(file_path, 'rb') as f:  
            parameters = pickle.loads(f.read())
        return parameters

In [34]:
class GRU4Rec(nn.Module):
    def __init__(self,parameter_path,input_length,embedding_dim,batch_size,num_layer=1):
        super(GRU4Rec,self).__init__()
        self.parameter_path = parameter_path
        self.input_length = input_length
        self.embedding_dim = embedding_dim
        self.num_layer = num_layer
        self.batch_size = batch_size
        self.emb = GetEmbedding(parameter_path,input_length,embedding_dim)
        self.gru_layers = nn.GRU(self.embedding_dim, self.embedding_dim,num_layers=self.num_layer, bias=False, batch_first=True)
        self.dense = nn.Linear(self.embedding_dim*2,embedding_dim)
        self.layer_norm = nn.LayerNorm(self.embedding_dim)
        self.apply(self.init_weights)


    def init_weights(self,module):
        if isinstance(module, nn.GRU):
            nn.init.xavier_uniform_(self.gru_layers.weight_hh_l0)
            nn.init.xavier_uniform_(self.gru_layers.weight_ih_l0)
    def forward(self,x_user_id_batch,x_item_seq_temp_batch,x_cat_list_batch,x_position_list_batch,x_item_seq_len_batch):
        
        user_id_emb = self.emb.user_id_embedding(x_user_id_batch)
        item_list_emb = self.emb.item_list_embedding(x_item_seq_temp_batch)
        category_list_emb = self.emb.category_list_embedding(x_cat_list_batch)
        position_list_emb = self.emb.position_list_embeddig(x_position_list_batch)
        behavior_list_emb = torch.cat([item_list_emb,category_list_emb],dim = 2)
        behavior_list_emb_dense = self.dense(behavior_list_emb)
        behavior_list_emb_dense = behavior_list_emb_dense+position_list_emb
        short_term_intent_temp,_ = self.gru_layers(behavior_list_emb_dense)
        short_term_intent_temp = self.gather_indexes(short_term_intent_temp,x_item_seq_len_batch)
        predict_behavior_emb = self.layer_norm(short_term_intent_temp)
        return predict_behavior_emb,self.emb.item_list_embedding_weight
    def gather_indexes(self,gru_output,x_item_seq_len_batch):
#       """Gathers the vectors at the specific positions over a minibatch."""
        mask_index = torch.reshape(x_item_seq_len_batch-2,[self.batch_size,1])
        flat_offsets = torch.reshape(torch.arange(self.batch_size) * self.input_length, [-1, 1])
        flat_positions = torch.reshape(mask_index + flat_offsets, [-1])
#         print(flat_positions)
        flat_sequence_tensor = torch.reshape(gru_output,
                                        [self.batch_size * self.input_length, self.embedding_dim])
#         print(flat_sequence_tensor.shape)
        output_tensor = flat_sequence_tensor.index_select(0,flat_positions)
#         print(output_tensor.shape)
        return output_tensor    

# 自定义loss

In [45]:
class Myloss(nn.Module):
    def __init__(self):
        super(Myloss,self).__init__()
        self.log_softmax = nn.LogSoftmax()
    def forward(self,emb,pred,truth):
        item_lookup_table_T = emb.t()
        logits = torch.matmul(pred,item_lookup_table_T)
        log_probs = self.log_softmax(logits)
        truth = torch.reshape(truth,[-1])
        one_hot_labels = F.one_hot(truth, num_classes=emb.shape[0])
        loss_origin = -torch.sum(log_probs.float() * one_hot_labels.float(), dim=-1)
        loss = torch.mean(loss_origin)
        return loss    

In [47]:
# train
batch_size = 256
parameter_path = 'data_example/parameters.pkl'
input_length = 50
embedding_dim = 128
num_layer = 1
gru4rec = GRU4Rec(parameter_path,input_length,embedding_dim,batch_size,num_layer)
gru4rec = gru4rec.to(device)
gru4rec.train()
criterion = Myloss()
train_loss = []
optimizer = optim.Adam(gru4rec.parameters(),lr=0.001)
for epoch in range(10):
    for batch_idx,data in enumerate(get_batch(user_id,item_seq_temp,cat_list,position_list,item_seq_len,target_item,batch_size)):
        x_user_id_batch = torch.LongTensor(data[0])
        x_item_seq_temp_batch = torch.LongTensor(data[1])
        x_cat_list_batch = torch.LongTensor(data[2])
        x_position_list_batch = torch.LongTensor(data[3])
        x_item_seq_len_batch = torch.LongTensor(data[4])
        y_batch = torch.LongTensor(data[5])
        predict_behavior_emb,item_list_emb_weight = gru4rec(x_user_id_batch,x_item_seq_temp_batch,x_cat_list_batch,x_position_list_batch,x_item_seq_len_batch)
#         print('item_list_emb.shape',item_list_emb_weight.shape)
        loss = criterion(item_list_emb_weight,predict_behavior_emb,y_batch)
        train_loss.append(loss.item())
        # Backward and optimizer
        # 1.优化器保存先前的梯度信息
        optimizer.zero_grad()
        # 2.计算梯度
        loss.backward()
#         3.梯度更新 w' = w - lr*grads
        optimizer.step()
        if batch_idx%200 == 0:
            print(epoch,batch_idx,loss.item())

d:\360downloads\software\anaconda3-5.0.1\anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


0 0 12.824366569519043
0 200 12.072864532470703
0 400 11.727947235107422
0 600 11.531489372253418


KeyboardInterrupt: 